In [3]:
import gymnasium as gym
from gymnasium import spaces
import numpy as np
import random
from stable_baselines3 import PPO
from stable_baselines3.common.env_checker import check_env
from stable_baselines3.common.vec_env import DummyVecEnv
import tkinter as tk
from tkinter import messagebox


class TTTBoard(object):
    def __init__(self):
        self.board = [0] * 9

    def get_board(self):
        return self.board

    def get_empty_pos(self):
        return [index for index, value in enumerate(self.board) if value == 0]

    def set_move(self, player_order, move):
        self.board[move] = player_order

    def is_over(self):
        board = self.board
        lines = [
            board[0:3], board[3:6], board[6:9],          # Filas
            board[0:9:3], board[1:9:3], board[2:9:3],    # Columnas
            [board[0], board[4], board[8]],              # Diagonal principal
            [board[2], board[4], board[6]]               # Diagonal secundaria
        ]
        for line in lines:
            if abs(sum(line)) == 3:
                return True, np.sign(sum(line))
        if 0 not in board:
            return True, 0  # Empate
        return False, None

    def __str__(self):
        symbols = {1: 'X', -1: 'O', 0: '.'}
        board = [symbols[value] for value in self.board]
        return f"{board[0]} | {board[1]} | {board[2]}\n--+---+--\n{board[3]} | {board[4]} | {board[5]}\n--+---+--\n{board[6]} | {board[7]} | {board[8]}\n"


In [4]:
class TicTacToeEnv(gym.Env):
    def __init__(self, opponent='random'):
        super(TicTacToeEnv, self).__init__()

        self.action_space = spaces.Discrete(9)
        self.observation_space = spaces.Box(low=-1, high=1, shape=(9,), dtype=np.int32)

        self.agent_player = 1  # El agente siempre es '1' (X)
        self.opponent = opponent  # Puede ser 'random' o 'human'
        self.human_move = None  # Variable para almacenar el movimiento del humano
        self.reset()

    def reset(self, seed=None, options=None):
        self.board = TTTBoard()
        self.current_player = random.choice([1, -1])  # Inicia aleatoriamente
        return np.array(self.board.get_board()), {}

    def step(self, action):
        if action is None:
            # Si no se proporciona una acción, simplemente retornamos el estado actual
            return np.array(self.board.get_board()), 0, False, False, {}

        if self.board.get_board()[action] != 0:
            # Penalizamos la acción inválida
            return np.array(self.board.get_board()), -2, False, False, {}

        # Movimiento del jugador actual (agente)
        self.board.set_move(self.current_player, action)

        # Verificamos si el juego ha terminado
        done, winner = self.board.is_over()
        if done:
            reward = self.calculate_reward(winner)
            return np.array(self.board.get_board()), reward, done, False, {}

        # Cambiamos al otro jugador
        self.current_player *= -1

        # Turno del oponente
        if self.current_player == -self.agent_player:
            if self.opponent == 'random':
                opponent_move = random.choice(self.board.get_empty_pos())
            elif self.opponent == 'human':
                # Obtenemos el movimiento del humano mediante Tkinter
                opponent_move = self.get_human_move_via_tkinter()
                if opponent_move is None:
                    # El humano cerró la ventana sin hacer un movimiento
                    print("El humano no realizó un movimiento. Fin del juego.")
                    done = True
                    reward = self.calculate_reward(self.agent_player)  # El agente gana
                    return np.array(self.board.get_board()), reward, done, False, {}
            else:
                raise ValueError("Tipo de oponente inválido")

            # Movimiento del oponente
            self.board.set_move(self.current_player, opponent_move)

            # Verificamos nuevamente si el juego ha terminado
            done, winner = self.board.is_over()
            if done:
                reward = self.calculate_reward(winner)
                return np.array(self.board.get_board()), reward, done, False, {}

            # Cambiamos de nuevo al agente
            self.current_player *= -1

        # No hay recompensa todavía
        return np.array(self.board.get_board()), 0, False, False, {}

    def calculate_reward(self, winner):
        if winner == self.agent_player:
            return 1  # Victoria
        elif winner == -self.agent_player:
            return -1  # Derrota
        else:
            return 0.5  # Empate

    def render(self):
        print(self.board)

    def close(self):
        pass

    def get_human_move_via_tkinter(self):
        # Crear la ventana de Tkinter
        root = tk.Tk()
        root.title("Tic Tac Toe - Turno del Humano")

        # Inicializamos 'human_move' como None
        self.human_move = None

        # Función callback para los botones
        def on_button_click(i):
            if self.board.get_board()[i] == 0:
                self.human_move = i
                root.destroy()
            else:
                messagebox.showwarning("Movimiento inválido", "Esa celda ya está ocupada.")

        # Crear botones para cada celda
        buttons = []
        for i in range(9):
            btn_text = ''
            state = 'normal'
            if self.board.get_board()[i] == 1:
                btn_text = 'X'
                state = 'disabled'
            elif self.board.get_board()[i] == -1:
                btn_text = 'O'
                state = 'disabled'
            button = tk.Button(root, text=btn_text, font=('normal', 60), width=2, height=1,
                               state=state,
                               command=lambda i=i: on_button_click(i))
            buttons.append(button)

        # Organizar los botones en una cuadrícula
        for i in range(9):
            row = i // 3
            col = i % 3
            buttons[i].grid(row=row, column=col)

        # Iniciar el bucle principal de Tkinter y esperar hasta que se cierre la ventana
        root.mainloop()

        return self.human_move


In [5]:
from stable_baselines3 import PPO
from stable_baselines3.common.env_checker import check_env

# Crear el entorno con el oponente aleatorio
env = TicTacToeEnv(opponent='random')

# Verificar el entorno
check_env(env)

# Crear y entrenar el modelo PPO
model = PPO("MlpPolicy", env, verbose=1)
model.learn(total_timesteps=1000000)

# Guardar el modelo entrenado
model.save("ppo_tictactoe_1M")

Using cuda device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 9.2      |
|    ep_rew_mean     | -9.63    |
| time/              |          |
|    fps             | 803      |
|    iterations      | 1        |
|    time_elapsed    | 2        |
|    total_timesteps | 2048     |
---------------------------------
-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 8.35        |
|    ep_rew_mean          | -8.33       |
| time/                   |             |
|    fps                  | 612         |
|    iterations           | 2           |
|    time_elapsed         | 6           |
|    total_timesteps      | 4096        |
| train/                  |             |
|    approx_kl            | 0.008421451 |
|    clip_fraction        | 0.0448      |
|    clip_range           | 0.2         |
|    entropy_loss  

In [6]:
from stable_baselines3.common.evaluation import evaluate_policy

# Evaluar el rendimiento del modelo entrenado
mean_reward, std_reward = evaluate_policy(model, env, n_eval_episodes=10)

print(f"Recompensa media: {mean_reward} +/- {std_reward}")

Recompensa media: 1.0 +/- 0.0


c:\Users\usuario\anaconda3\envs\env_101\Lib\site-packages\stable_baselines3\common\evaluation.py:67: UserWarning: Evaluation environment is not wrapped with a ``Monitor`` wrapper. This may result in reporting modified episode lengths and rewards, if other wrappers happen to modify these. Consider wrapping environment first with ``Monitor`` wrapper.
  warnings.warn(


In [1]:
# env.opponent = 'human'
# model.learn(total_timesteps=1)

# Test and train

In [7]:
import tkinter as tk
from tkinter import messagebox

def play_against_human_ui(model, env):
    # Inicializar el estado del juego
    obs, _ = env.reset()
    done = False

    # Crear la ventana principal
    root = tk.Tk()
    root.title("Tres en Raya - Juega contra el Agente")

    # Crear una lista para almacenar los botones
    buttons = []

    # Función para actualizar el tablero en la interfaz gráfica
    def update_board():
        env.render()
        board = env.board.get_board()
        symbols = {1: 'X', -1: 'O', 0: ' '}
        for i in range(9):
            buttons[i]['text'] = symbols[board[i]]
            if board[i] != 0:
                buttons[i]['state'] = 'disabled'
            else:
                buttons[i]['state'] = 'normal'

    # Función para manejar el clic del usuario
    def on_button_click(i):
        nonlocal obs, done
        if not done and env.current_player == -env.agent_player:
            action = i
            if action in env.board.get_empty_pos():
                print(f"El humano juega en la posición {action + 1}")
                obs, reward, done, _, _ = env.step(action)
                update_board()
                if done:
                    end_game(reward)
                else:
                    # Turno del agente
                    root.after(100, agent_turn)
            else:
                messagebox.showwarning("Movimiento inválido", "Esta posición ya está ocupada.")

    # Función para el turno del agente
    def agent_turn():
        nonlocal obs, done
        if not done and env.current_player == env.agent_player:
            while True:
                action, _states = model.predict(obs, deterministic=True)
                obs, reward, done, _, _ = env.step(action)
                if reward == -2 and not done:
                    # El agente hizo una acción inválida, intentará de nuevo
                    print(f"El agente intentó una acción inválida en la posición {action + 1}. Intentando de nuevo.")
                    continue
                else:
                    print(f"El agente juega en la posición {action + 1}. R: {reward}")
                    update_board()
                    if done:
                        end_game(reward)
                    break

    # Función para manejar el final del juego
    def end_game(reward):
        if reward == 1:
            messagebox.showinfo("Fin del juego", "¡El agente ha ganado!")
        elif reward == -1:
            messagebox.showinfo("Fin del juego", "¡Has ganado!")
        else:
            messagebox.showinfo("Fin del juego", "Es un empate.")
        
        # Esto no tiene efecto, ya que está pegado al entorno random: model.learn(total_timesteps=1)
        root.destroy()

    # Crear los botones del tablero
    for i in range(9):
        button = tk.Button(root, text=' ', font=('Helvetica', 32), width=5, height=2,
                           command=lambda i=i: on_button_click(i))
        button.grid(row=i // 3, column=i % 3)
        buttons.append(button)

    # Comenzar el juego: si el agente inicia, realiza su movimiento
    if env.current_player == env.agent_player:
        agent_turn()

    # Actualizar el tablero inicialmente
    update_board()

    # Ejecutar la interfaz gráfica
    root.mainloop()

In [32]:
def play_against_human(model, env):
    obs, _ = env.reset()
    done = False
    env.render()
    while not done:
        if env.current_player == env.agent_player:
            # Turno del agente
            while True:
                action, _states = model.predict(obs, deterministic=True)
                obs, reward, done, _, info = env.step(action)
                if reward == -2 and not done:
                    # El agente hizo una acción inválida, lo intentará de nuevo
                    print(f"El agente intentó una acción inválida en la posición {action + 1}. Intentando de nuevo.")
                    # Opcionalmente, puedes hacer que el agente aprenda de este error
                    continue  # El agente vuelve a intentar
                else:
                    print(f"El agente juega en la posición {action + 1}. R: {reward}")
                    env.render()
                    break  # El agente realizó un movimiento válido
        else:
            # Turno del humano
            valid_move = False
            while not valid_move:
                try:
                    action = int(input("Elige una posición (1-9): ")) - 1
                    if action in env.board.get_empty_pos():
                        valid_move = True
                    else:
                        print(f"Movimiento inválido. Intenta nuevamente. Action selected: {action}")
                except ValueError:
                    print("Entrada inválida. Por favor ingresa un número del 1 al 9.")
                    
            print(f"El humano juega en la posición {action + 1}")
            obs, reward, done, _, info = env.step(action)
            env.render()
        if done:
            env.render()
            if reward == 1:
                print("¡El agente ha ganado!")
            elif reward == -1:
                print("¡Has ganado!")
            else:
                print("Es un empate.")
            # Actualizamos el modelo para que continúe aprendiendo
            model.learn(total_timesteps=1)
            break

In [9]:
# Cargar el modelo entrenado
model = PPO.load("ppo_tictactoe_1M", env=env)

# Crear un nuevo entorno para jugar contra el humano
human_env = TicTacToeEnv(opponent='human')

# Jugar contra el humano y continuar aprendiendo
play_against_human_ui(model, human_env)

Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
El humano juega en la posición 9
. | . | .
--+---+--
. | . | .
--+---+--
. | X | O

El humano juega en la posición 6
. | . | .
--+---+--
. | X | O
--+---+--
. | X | O

El agente juega en la posición 2. R: 1
. | X | .
--+---+--
. | X | O
--+---+--
. | X | O

El agente juega en la posición 5. R: 1
. | X | .
--+---+--
. | X | X
--+---+--
. | X | O

El agente juega en la posición 8. R: 1
. | X | .
--+---+--
. | X | X
--+---+--
. | X | O



Exception in Tkinter callback
Traceback (most recent call last):
  File "c:\Users\usuario\anaconda3\envs\env_101\Lib\tkinter\__init__.py", line 1967, in __call__
    return self.func(*args)
           ^^^^^^^^^^^^^^^^
  File "c:\Users\usuario\anaconda3\envs\env_101\Lib\tkinter\__init__.py", line 861, in callit
    func(*args)
  File "C:\Users\usuario\AppData\Local\Temp\ipykernel_25016\888047381.py", line 58, in agent_turn
    update_board()
  File "C:\Users\usuario\AppData\Local\Temp\ipykernel_25016\888047381.py", line 22, in update_board
    buttons[i]['text'] = symbols[board[i]]
    ~~~~~~~~~~^^^^^^^^
  File "c:\Users\usuario\anaconda3\envs\env_101\Lib\tkinter\__init__.py", line 1732, in __setitem__
    self.configure({key: value})
  File "c:\Users\usuario\anaconda3\envs\env_101\Lib\tkinter\__init__.py", line 1721, in configure
    return self._configure('configure', cnf, kw)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\usuario\anaconda3\envs\env_101\Lib\tkinter\

TclError: invalid command name ".!button"

In [33]:
# Cargar el modelo entrenado
model = PPO.load("ppo_tictactoe", env=env)

# Crear un nuevo entorno para jugar contra el humano
human_env = TicTacToeEnv(opponent='human')

# Jugar contra el humano y continuar aprendiendo
play_against_human(model, human_env)

Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
. | . | .
--+---+--
. | . | .
--+---+--
. | . | .

El humano juega en la posición 1
O | . | .
--+---+--
. | . | .
--+---+--
. | . | .

El agente juega en la posición 7. R: 0
O | . | .
--+---+--
. | . | .
--+---+--
X | . | .

El humano juega en la posición 2
O | O | .
--+---+--
. | . | .
--+---+--
X | . | .

El agente juega en la posición 5. R: 0
O | O | .
--+---+--
. | X | .
--+---+--
X | . | .

Movimiento inválido. Intenta nuevamente. Action selected: 1
El humano juega en la posición 4
O | O | .
--+---+--
O | X | .
--+---+--
X | . | .

El agente juega en la posición 3. R: 1
O | O | X
--+---+--
O | X | .
--+---+--
X | . | .

O | O | X
--+---+--
O | X | .
--+---+--
X | . | .

¡El agente ha ganado!
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 3.89     |
|    ep_rew_mean     | 0.565    |
| time/              |          |
|    fps             | 643      |
|    iteratio